In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
import glob
import torch
import os
from numpy.fft import fft, ifft, fftshift, ifftshift
import pickle
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline
plt.rcParams['figure.figsize'] = 9, 6

In [ ]:
from torch_echo.demodulators.demodulator_classic import DemodulatorClassic
from torch_echo.modulators.modulator_neural import ModulatorNeural
from torch_echo.modulators.modulator_classic import ModulatorClassic
from torch_echo.utils.util_data import bits_to_integers, integers_to_bits

In [ ]:
import echoecho.EchoPacketWrapper as EPW
import echoecho.DSPUtil as dsp

In [ ]:
def db20(x):
    return 20 * np.log10(np.abs(x))

In [ ]:
HOME = os.environ["HOME"]
os.chdir(os.path.join(HOME,'tmp/snr-runs'))

## Get SNR-run map

In [ ]:
def get_ber(file):
    with open(file, "r") as f:
        nsamps = int(f.readline().strip().split()[-2])
        ber = float(f.readline().strip().split()[-1])
    return nsamps, ber

In [ ]:
dirs = glob.glob('*-srn1')  # Only SRN1 recorded IQ
gain_dirs = {}
gain_bers = {}
for d in dirs:
    with open(d + "/tx-gain", "r") as f:
        gain = float(f.read().strip())
        gain_dirs[gain] = d
        _, ber = get_ber(d + "/results")
        gain_bers[gain] = ber
print(gain_dirs)
print(gain_bers)

In [ ]:
gains = sorted(gain_dirs.keys())
bers = [gain_bers[g] for g in gains]
plt.plot(gains, bers, label='Classic BER')
plt.plot([6,16],[.01,.01], linestyle='--', label='1% BER (Training SNR)')
plt.legend()
plt.grid()
plt.xlabel('TX Gain dB')
plt.ylabel('Classic QPSK BER')
plt.yscale('log')
plt.title("Gain vs BER for Classics")

In [ ]:
epw = EPW.EchoPacketWrapper(beta_rrc=0.13, corr_repetitions=2, samps_per_symb=2, cfo_freqs=[0.25], cfo_samps=1)

In [ ]:
bps = 2
sps = 2
pktsz = int(epw.full_packet_length((512 * 2 + 128 + 96) / bps))
iqsz = int(pktsz * sps)
pktsz

In [ ]:
def extract_pkts(iq, step=20000, iqsize=iqsz, pktsize=pktsz):
    noise = []
    raw_pkts = []
    pkts = []
    while iq.size > iqsize and len(pkts) < 100:
        offset = epw.find_channel_estimate_field(iq[:step], 50)
        if offset is not None:
            noise.append(iq[:offset])
            raw_pkts.append(iq[offset:offset + iqsz])
            pkts.append(dsp.rrc_decimate_fft(raw_pkts[-1], 0.13, 2))
            iq = iq[offset + iqsz:]
        else:
            iq = iq[step:]
    return raw_pkts, pkts, noise

In [ ]:
gain_snrs = {}
for g in gains:
    gdir = gain_dirs[g]
    iq = np.fromfile(gdir + "/iq-in.bin", dtype=np.complex64)
    raws, pkts, noise = extract_pkts(iq)
    noise_pwrs = []
    body_pwrs = []
    for i in range(len(pkts)):
        p = pkts[i]
        n = noise[i]
        noise_pwrs.append(np.median(db20(dsp.rrc_decimate_fft(n, 0.13, 2))))
        body_pwrs.append(np.median(db20(p[600:1100])))
    snr = np.mean(body_pwrs) - np.mean(noise_pwrs)
    gain_snrs[g] = snr
print(gain_snrs)

In [ ]:
gain_snrs2 = {}
for g in gains:
    gdir = gain_dirs[g]
    iq = np.fromfile(gdir + "/iq-in.bin", dtype=np.complex64)
    raws, pkts, noise = extract_pkts(iq)
    noise_pwrs = []
    body_pwrs = []
    for i in range(len(pkts)):
        p = raws[i]
        n = noise[i]
        noise_pwrs.append(np.median(db20(n)))
        body_pwrs.append(np.median(db20(p[1200:2200])))
    snr = np.mean(body_pwrs) - np.mean(noise_pwrs)
    gain_snrs2[g] = snr
print(gain_snrs2)

In [ ]:
plt.plot(gains, [gain_snrs[g] for g in gains], label='Results')
plt.plot(list(range(6,17)), np.arange(6, 17)+0.5, linestyle='--', label='1-to-1')
plt.plot([10.5, 10.5], [7, 16], linestyle='--', label='1% BER Gain')
plt.annotate("({}, {:.1f})".format(10.5, gain_snrs[10.5]), (10.75, gain_snrs[10.5]-0.25))
plt.legend()
plt.grid()
plt.xlabel("TX Gain dB")
plt.ylabel("RX SNR dB")
plt.title("TX Gain vs Achieved SNR @ 1 Samples Per Symbol")

In [ ]:
plt.plot(gains, [gain_snrs2[g] for g in gains], label='Results')
plt.plot(list(range(6,17)), list(range(5, 16)), linestyle='--', label='1-to-1')
plt.plot([10.5, 10.5], [5, 15], linestyle='--', label='1% BER Gain')
plt.annotate("({}, {:.1f})".format(10.5, gain_snrs2[10.5]), (10.75, gain_snrs2[10.5]-0.25))
plt.legend()
plt.grid()
plt.xlabel("TX Gain dB")
plt.ylabel("RX SNR dB")
plt.title("TX Gain vs Achieved SNR @ 2 Samples Per Symbol")

In [ ]:
plt.plot([gain_snrs[g] for g in gains], bers, label='Classic BER')
plt.plot([6,16],[.01,.01], linestyle='--', label='1% BER (Training SNR)')
plt.legend()
plt.grid()
plt.xlabel('SNR dB')
plt.ylabel('Classic QPSK BER')
plt.yscale('log')
plt.title("SNR vs BER for Classics")

# BER vs Gain vs SNR 

In [ ]:
mygains = np.array([6, 10.5, 13, 14.5, 16])
gains = np.array(gains)

In [ ]:
snrs = np.array([gain_snrs[g] for g in gains])
bers = np.array(bers)
msnr, bsnr = np.polyfit(gains, snrs, 1)
linear = lambda x: msnr * x + bsnr
coeffs = np.polyfit(snrs, np.log(bers), 2)
quadr = lambda x: np.exp(coeffs[0] * x ** 2 + coeffs[1] * x + coeffs[2])

In [ ]:
plt.plot(gains, [gain_snrs[g] for g in gains], label='Results')
plt.plot(gains, linear(gains))
plt.plot(list(range(6,17)), list(range(5, 16)), linestyle='--', label='1-to-1')
plt.plot([10.5, 10.5], [5, 15], linestyle='--', label='1% BER Gain')
plt.annotate("({}, {:.1f})".format(10.5, gain_snrs2[10.5]), (10.75, gain_snrs2[10.5]-0.25))
plt.legend()
plt.grid()
plt.xlabel("TX Gain dB")
plt.ylabel("RX SNR dB")
plt.title("TX Gain vs Achieved SNR @ 2 Samples Per Symbol")
plt.show()

plt.plot([gain_snrs[g] for g in gains], bers, label='Classic BER')
plt.plot(np.arange(6, 16, 0.5), quadr(np.arange(6, 16, 0.5)))
plt.plot([6,15],[.01,.01], linestyle='--', label='1% BER (Training SNR)')
plt.legend()
plt.grid()
plt.xlabel('SNR dB')
plt.ylabel('Classic QPSK BER')
plt.yscale('log')
plt.title("SNR vs BER for Classics")

In [ ]:
mysnrs = linear(mygains)
print(mysnrs)
mybers = quadr(mysnrs)
print(mybers)

In [ ]:
out = {'gain-snr-linear': [msnr, bsnr],
       'snr-ber-logquadratic': list(coeffs)}
with open("gain-snr-ber-coeffs.pkl", "wb") as f:
    pickle.dump(out, f)